In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from jinja2 import Template
from math import sqrt, pi
from scipy.optimize import brentq
import glob, json
import os
try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd $workdir

In [ ]:
def d_AH(m,salt='NaCl'):
    # density of alkali halides NaCl, NaI, KCl
    # expression from Laliberté et al. DOI: 10.1021/je0498659
    wt = m*df_props[salt]['mw'] # g solute / kg solvent
    wtFrac = wt / (wt + 1000) # weight fraction
    T = 25
    c0, c1, c2, c3, c4 = df_props[salt]['d']['const']
    A = (wtFrac+c2+c3*T)/(c0*wtFrac+c1)/np.exp(1e-6*(T+c4)**2)
    return 1. / ( A*wtFrac + (1-wtFrac) / d_hoh(T) )/1000

def d_hoh(T):
    # density of water Laliberté et al. DOI: 10.1021/je0498659
    nom = (((((-2.8054253*1e-10*T+1.0556302*1e-7)*T-4.6170461*1e-5)*T-0.0079870401)*T+16.945176)*T+999.83952)
    den = 1+0.01687985*T
    return nom/den

def M2m(M,salt='NaCl'): 
    mw = df_props[salt]['mw']
    return brentq(lambda x : x - M/(df_props[salt]['d']['func'](x,salt) - M*mw/1000), 0, 100) 

def m2M(m,salt='NaCl'):
    mw = df_props[salt]['mw']
    return m/(1+m*mw/1000)*df_props[salt]['d']['func'](m,salt)

#MOLALITY
m_nacl = np.array([.1,.15,.2,.25,.3,.35,.4,.45,.5,.55,.6,.65,.7,.75,.8,.85,.9,.95,1.0,1.1,1.2,1.4,1.6,1.8,2.0,2.5,3.0,3.5,4.0])

#ACTIVITY COEFF
g_nacl = np.array([.778,.750,.735,.720,.710,.700,.693,.686,.681,.677,.673,.670,.667,.664,.662,.660,.659,.658,.657,.655,.654,.655,.657,.662,.668,.688,.714,.746,.783])

# properties of the salt
df_props = pd.DataFrame({'NaCl':{'mw':58.40,'ActCoef':{'g':g_nacl,'m':m_nacl},'M2m':M2m,
                            'd':{'func':d_AH,'const':[-0.00433,0.06471,1.01660,0.014624,3315.6]}}})

# function to obtain activity coefficients as a function of salt molality
def g_func(m, b1=1.4369, b2=0.0054, b3=0.0495, b4=0.0092):
    return np.exp( -1.18*np.sqrt(m)/(1+b1*np.sqrt(m)) - np.log(1-b2*m) + b3*m + b4*m**2 )

plt.plot(m_nacl,g_nacl, lw=0, marker='o')
plt.plot(m_nacl,g_func(m_nacl))
plt.ylabel('Activity Coefficient')
plt.xlabel('Molality  /  mol kg$^{-1}$')

In [ ]:
cation = pd.Series({3:158.2, 4:165.4, 5:170.6, 5.8:174.18, 6.6:179.2, 7.4:186.4, 8:189})
anion = pd.Series({3:157.94, 4:161.47, 5:160.64, 5.8:160.56, 6.6:157.97, 7.4:152.74, 8:150.83})
dianion = pd.Series({3:0.19, 4:1.92, 5:3.54, 5.8:2.82, 6.6:6.79, 7.4:15.44, 8:18.61})
trianion = pd.Series({3:7.6e-5, 4:0.04, 5:0.96, 5.8:2.66, 6.6:2.52, 7.4:0.9, 8:0.28})
pHsalt = pd.DataFrame({'cation':cation, 'anion':anion, 'dianion':dianion, 'trianion':trianion})*1e-3
# Convert molarity of cations and anions to activity using NaCl activity coefficients
pHsalt['cation'] = pHsalt['cation'].apply(lambda x : g_func(M2m(x))*x)
pHsalt['anion'] = pHsalt['anion'].apply(lambda x : g_func(M2m(x))*x)
pHsalt

In [ ]:
T = 298 # K
R = 8.3145 # J/K/mol
eps = 2.5 # kJ/mol
print("eps:",eps,"kJ/mol")
fepsw = lambda T : 5321/T+233.76-0.9297*T+0.1417*1e-2*T*T-0.8292*1e-6*T**3
epsw = 78 # fepsw(T)
lB = 1.60217662**2/(4*np.pi*8.8541878*epsw*1.38064852*T)*1e7
print(lB,epsw)

In [ ]:
L = pd.Series({'HD':2.5,'T1':3.5,'T2':4.1})
M = pd.Series({'HD':2.2,'T1':3.5,'T2':4.2})
S = pd.Series({'HD':1.8,'T1':3.3,'T2':4.4})
models = pd.DataFrame({'L':L,'M':M,'S':S})
models

In [ ]:
templateRNA = Template("""comment: "3-bead Cooke lipid model. For more information: doi:10/chqzjk"
temperature: {{T}}
random: {seed: hardware}
geometry: {type: hexagonal, length: {{L}}, radius: {{R}}}
mcloop: {macro: 10, micro: {{micro}}}

atomlist:
    - HD:   {sigma: {{RHD*2}}, eps: {{eps}}, dp: 2}
    - HHD:  {sigma: {{RHD*2}}, eps: {{eps}}, dp: 2, q: 1}
    - T1:  {sigma: {{RT1*2}}, eps: {{eps}}, dp: 2}
    - T2:  {sigma: {{RT2*2}}, eps: {{eps}}, dp: 2, alphax: -18}
    - NA:  {sigma: 4.6, eps: 0.01, dp: 10, q: 1}
    - CL:  {sigma: 4.6, eps: 0.01, dp: 10, q: -1}
    - H:   {pactivity: {{pH}}, eps: 0.01, implicit: True}

moleculelist:
    - Na: {atoms: [NA], atomic: True, activity: {{pHsalt.loc[pH,'cation']}}}
    - Cl: {atoms: [CL], atomic: True, activity: {{pHsalt.loc[pH,'anion']}}}
    - lipid:
        activity: {{activity}}
        structure:
            - H:  [0,0,0]
            - T1: [0,0,{{RHD+RT1}}]
            - T2: [0,0,{{RHD+2*RT1+RT2}}]
        bondlist:
            - fene: {index: [0,1], k: 1.2, rmax: {{1.5*(RHD+RT1)}}}
            - fene: {index: [1,2], k: 1.2, rmax: {{1.5*(RT1+RT2)}}}
            - harmonic: {index: [0,2], k: 0.4, req: {{2.5*RHD+3*RT1+2.5*RT2}}}
        keeppos: True
            
insertmolecules:
    - Na: {N: 400, inactive: True}
    - Cl: {N: 400, inactive: True}
    - lipid: {N: 400, inactive: True}

energy:
    - bonded: {}
    - nonbonded_splined:
        default:
            - wca:
                mixing: LB
            - custom:
                cutoff: 1000
                function: lB * q1 * q2 * ( 1.0 / r - 1.0 / Rc )
                constants:
                    lB: {{lB}}
            - polar: {epsr: {{epsw}}}
        T1 T1:
            - wca:
                mixing: LB
            - cos2: {rc: {{1.122462*2*RT1}}, eps: {{eps}}, wc: {{wc*2*RT1}}}
        T1 T2:
            - wca:
                mixing: LB
            - cos2: {rc: {{1.122462*(RT1+RT2)}}, eps: {{eps}}, wc: {{wc*(RT1+RT2)}}}
        T2 T2:
            - wca:
                mixing: LB
            - cos2: {rc: {{1.122462*2*RT2}}, eps: {{eps}}, wc: {{wc*2*RT2}}}

reactionlist:
    - "HHD = HD + Na + H": {pK: {{pKa}}}
    - "HHD + Cl = HD + H": {pK: {{pKa}}}
    - "= Na + Cl": {}
    - "= lipid": {neutral: True}

moves:
    - rcmc: {repeat: {{repeat}}}
    - moltransrot: {molecule: lipid, dp: 0.2, dprot: 0.5, repeat: {{repeat}}}
    - transrot: {molecule: lipid, repeat: {{repeat}}}
    - transrot: {molecule: Cl, repeat: {{repeat}}}
    - transrot: {molecule: Na, repeat: {{repeat}}}
    - volume: {dV: 0.05, method: isochoric, repeat: 1}
    
analysis:
    - savestate: {file: state.json}
    - savestate: {file: confout.pqr}
    - savestate: {file: confout.gro}
    - savestate: {file: cuboid.pqr, convert_hexagon: True, nstep: 4e5}
    - xtcfile: {file: traj.xtc, nstep: 1e4}
    - qrfile: {file: qrtraj.dat, nstep: 1e4}
    - atomrdf: {file: rdf_HHD_HHD.dat, nstep: 1e3, name1: 'HHD', name2: 'HHD', dr: 0.1, dim: 1, slicedir: [0,0,1], thickness: {{RHD}}}
    - atomrdf: {file: rdf_HHD_HD.dat, nstep: 1e3, name1: 'HHD', name2: 'HD', dr: 0.1, dim: 1, slicedir: [0,0,1], thickness: {{RHD}}}
    - atomrdf: {file: rdf_HD_HHD.dat, nstep: 1e3, name1: 'HD', name2: 'HHD', dr: 0.1, dim: 1, slicedir: [0,0,1], thickness: {{RHD}}}
    - atomrdf: {file: rdf_HD_HD.dat, nstep: 1e3, name1: 'HD', name2: 'HD', dr: 0.1, dim: 1, slicedir: [0,0,1], thickness: {{RHD}}}
    - reactioncoordinate: {file: order.dat, nstep: 1e3, type: system, property: OrderParam, index: 2}
    - reactioncoordinate: {file: NNa.dat, nstep: 1e3, type: molecule, property: N, index: 0}
    - reactioncoordinate: {file: NCl.dat, nstep: 1e3, type: molecule, property: N, index: 1}
    - reactioncoordinate: {file: Nlipids.dat, nstep: 1e3, type: atom, property: N, index: 2}
    - reactioncoordinate: {file: radius.dat, nstep: 1e3, type: system, property: radius}
    - reactioncoordinate: {file: Lz.dat, nstep: 1e3, type: system, property: Lz}
    - reactioncoordinate: {file: charge.dat, nstep: 1e3, type: system, property: Q}
    - reactioncoordinate: {file: NHHD.dat, nstep: 1e3, type: atom, property: N, index: 1}
    - reactioncoordinate: {file: Rcyl.dat, nstep: 1e3, type: molecule, property: Rinner, indexes: [2,3,0,1], dir: [1,1,0]}
    - reactioncoordinate: {file: RCl.dat, nstep: 1e3, type: molecule, property: Rinner, indexes: [2,3,5,5], dir: [1,1,0]}
    - atomprofile: {file: distHead.dat, nstep: 1e3, atoms: [HD,HHD], dir: [1,1,0], atomcom: T1, dr: 0.2}
    - atomprofile: {file: distT1.dat, nstep: 1e3, atoms: [T1], dir: [1,1,0], atomcom: T1, dr: 0.2}
    - atomprofile: {file: distT2.dat, nstep: 1e3, atoms: [T2], dir: [1,1,0], atomcom: T1, dr: 0.2}
    - atomprofile: {file: distCL.dat, nstep: 1e3, atoms: [CL], dir: [1,1,0], atomcom: T1, dr: 0.2}
    - systemenergy: {file: energy.dat, nstep: 1e5}
""")

In [ ]:
submit = Template("""#!/bin/bash
#requesting the number of cores needed on exclusive nodes
#SBATCH -N 1
#SBATCH --ntasks-per-node=1
#SBATCH -A lu2022-2-36
#
# job time, change for what your job requires
#SBATCH -t 100:0:0
#
# job name
#SBATCH -J {{dirname}}
#
# filenames stdout and stderr
#SBATCH -o out
#SBATCH -e err

source /home/gtesei00/.bashrc
module load GCCcore/9.3.0
module load CMake/3.16.4
module load GCC/9.3.0
conda activate faunus

/home/gtesei00/Jun22/faunus/faunus --verbosity 0 --input inp.json --output outp.json --state state.json 
""")

In [ ]:
%cd $workdir
faunus_path = '/home/gtesei00/Jun22/faunus'
dirname = '{:s}_{:.2f}_{:.1f}_{:.2g}'
Nnuc = 72
Lf = 48; L = 7.2*Nnuc; V = 2*np.sqrt(3)*31*31*Lf; R = np.sqrt( V / L / 2 / np.sqrt(3) )
pKa = 9.41
for m in ['L','S','M']:
    for pH in pHsalt.index.values[:3]:
        aL = 3e-3/(1+10**(pKa-pH))
        #!cp {dirname.format('L',pKa,pH,aL)}/state.json {dirname.format(m,pKa,pH,aL)}
        if not os.path.isdir(dirname.format(m,pKa,pH,aL)):
            !cp -r {dirname.format('L',pKa,pH,aL)} {dirname.format(m,pKa,pH,aL)}
        else:
            print('isdir')
        #!cp /proj/snic2021-23-45/users/x_giute/large13/{dirname.format('S',pKa,pH,aL)}/state.json {dirname.format(m,pKa,pH,aL)}
        %cd {dirname.format(m,pKa,pH,aL)}
        with open('inp.yml', 'w') as input_file:
            input_file.write(templateRNA.render(T=T, wc=1.8, L=L, R=R, micro=4e5,
                        RHD=models.loc['HD',m], RT1=models.loc['T1',m],
                        RT2=models.loc['T2',m], repeat=10, pKa=pKa,
                        pHsalt=pHsalt, Nnuc=Nnuc, pH=pH, epsw=epsw, lB=lB, eps=eps,
                        activity=aL))
        !{faunus_path}/scripts/yason.py inp.yml > inp.json
        with open('submit.sh', 'w') as submit_file:
            temp = submit.render(dirname=dirname.format(m,pKa,pH,aL))
            submit_file.write(temp)
        !sbatch submit.sh
        %cd ..

In [ ]:
%cd $workdir
faunus_path = '/home/gtesei00/Jun22/faunus'
dirname = '{:s}_{:.2f}_{:.1f}_{:.2g}'
Nnuc = 72
Lf = 48; L = 7.2*Nnuc; V = 2*np.sqrt(3)*31*31*Lf; R = np.sqrt( V / L / 2 / np.sqrt(3) )
pKa = 8.46
for m in ['L']:
    for pH in pHsalt.index.values[:1]:
        aL = 3e-3/(1+10**(pKa-pH))
        aL_1 = 3e-3/(1+10**(9.41-pH))
        #!cp ../lnps_norna/{dirname.format(m,pKa,pH,aL)}/state.json {dirname.format(m,pKa,pH,aL)}
        if not os.path.isdir(dirname.format(m,pKa,pH,aL)):
            !cp -r {dirname.format('L',8.46,pH,aL)} {dirname.format(m,pKa,pH,aL)}
        else:
            print('isdir')
        #!cp /proj/snic2021-23-45/users/x_giute/large/{dirname.format('L',pKa,pH,aL)}/state.json {dirname.format(m,pKa,pH,aL)}
        %cd {dirname.format(m,pKa,pH,aL)}
        with open('inp.yml', 'w') as input_file:
            input_file.write(templateRNA.render(T=T, wc=1.8, L=L, R=R, micro=4e5,
                        RHD=models.loc['HD',m], RT1=models.loc['T1',m],
                        RT2=models.loc['T2',m], repeat=10, pKa=pKa,
                        pHsalt=pHsalt, Nnuc=Nnuc, pH=pH, epsw=epsw, lB=lB, eps=eps,
                        activity=aL))
        !{faunus_path}/scripts/yason.py inp.yml > inp.json
        with open('submit.sh', 'w') as submit_file:
            temp = submit.render(dirname=dirname.format(m,pKa,pH,aL))
            submit_file.write(temp)
        !sbatch submit.sh
        %cd ..